In [6]:
#descargar paquete
# https://github.com/UB-Mannheim/tesseract/wiki
#  en la linea
""" The latest installer can be downloaded here:
tesseract-ocr-w64-setup-5.3.3.20231005.exe (64 bit)"""

#carpeta de mi destino
#C:\Program Files\Tesseract-OCR


' The latest installer can be downloaded here:\ntesseract-ocr-w64-setup-5.3.3.20231005.exe (64 bit)'

In [7]:
import cv2
import pytesseract
import numpy as np

pytesseract.pytesseract.tesseract_cmd=r"C:\Program Files\Tesseract-OCR\tesseract.exe"

In [8]:
def eliminar_puntos_pequenos(img):
    # Binarizar la imagen (convertir a blanco y negro)
    _, img_binaria = cv2.threshold(img, 128, 255, cv2.THRESH_BINARY)
    kernel = np.ones((3, 3), np.uint8)

    # Aplicar la erosión para eliminar puntos pequeños
    img = cv2.erode(img_binaria, kernel, iterations=3)

    # Mostrar la imagen original y la imagen sin puntos pequeños
    cv2.imshow('Imagen Original', img_binaria)
    cv2.imshow('Imagen Sin Puntos Pequeños', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return img

def ajustar_relacion_aspecto(img):
    altura, anchura = img.shape[:2]
    print(float(anchura/altura))
    if float(anchura/altura) < 1.5:
        # Calcular la nueva altura según la fórmula
        nueva_altura = int(((anchura * 2) / 3))

        # Calcular cuántos píxeles quitar de la parte superior e inferior
        pixeles_a_quitar = (altura - nueva_altura) // 2
        img_recortada = img[pixeles_a_quitar:altura - pixeles_a_quitar, :]

        cv2.imshow('Imagen Original', img)
        cv2.imshow('Imagen Recortada', img_recortada)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        return img_recortada
    return img
    

def eliminar_manchas(imagen_path):
    # Definir el kernel para las operaciones morfológicas
    kernel = np.ones((5, 5), np.uint8)
    
    img = cv2.imread(imagen_path, cv2.IMREAD_GRAYSCALE)
    img = eliminar_puntos_pequenos(img)
    img = ajustar_relacion_aspecto(img)

    # Aplicar la operación de apertura
    Plate_Cleaned = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
    
    #show changes
    cv2.imshow('Imagen Original', img)
    cv2.imshow('Imagen Filtrada', Plate_Cleaned)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return Plate_Cleaned

#your number th cars
i=4
Plate_Cleaned = eliminar_manchas(f'./Plates_Img/{i}.png')


1.9245283018867925


In [9]:
def extract_num(img):
    # Obtener las dimensiones originales de la imagen
    alto, ancho = img.shape[:2]

    # Calcular el 5% de las dimensiones originales
    porcentaje_recorte = 0.02
    recorte_ancho = round(ancho * porcentaje_recorte)
    recorte_alto = round(alto * porcentaje_recorte)

    # Definir las regiones de interés (ROI) para recortar
    roi_superior = recorte_alto
    roi_inferior = alto - recorte_alto
    roi_izquierda = recorte_ancho
    roi_derecha = ancho - recorte_ancho

    # Recortar la imagen utilizando las regiones de interés
    img = img[roi_superior:roi_inferior, roi_izquierda:roi_derecha]

    cv2.imshow('Placa Marcada', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    text = pytesseract.image_to_string(img, config='--psm 11')
    return text
print(extract_num(Plate_Cleaned))

In [10]:
# cv2.imshow('Placa Marcada', placa)
# cv2.waitKey(0)
# cv2.destroyAllWindows()